## `2.4 Ссылочная модель данных`

`Практический пример: ` <br>
в банке, где мы работаем, клиент может установить лимит трат на месяц. 

Пусть у нас есть список уже понесённых трат за месяц. 

Выставим проверку на каждую покупку клиента: в момент совершения покупки мысленно добавим ее сумму к списку трат, подсчитаем сумму и сравним с лимитом. 

Если получилось меньше — даем добро, иначе отклоняем. Подчеркнем: пока что покупка не совершена, это только прикидка.

In [9]:
# сумма покупки, список покупок, лимит
def can_purchase (amount, history, limit):
    # добавляем покупку в history
    history.append(amount)
    # Просуммируем все элементы history, сравним с limit
    # Вернём True или False
    return sum(history) <= limit

limit = 100
history = [50, 40]

# 90 + 4 должно вернуться True - так и есть
print(can_purchase(4, history, limit))
# 90 + 7 должно вернуться True - но возвращается False
print(can_purchase(7, history, limit))


True
False


In [10]:
# распечатаем список покупок
def can_purchase (amount, history, limit, do_print = False):
    history.append(amount)
    if do_print:
        print(history)
    return sum(history) <= limit

limit = 100
history = [50, 40]

# Аргументы можно передавать по имени: явно говорим, что do_print будет равно True
print(can_purchase(4, history, limit, do_print = True))
print(can_purchase(7, history, limit, do_print = True))

# Во втором вызове функции список содержал покупку из первого!

[50, 40, 4]
True
[50, 40, 4, 7]
False


In [11]:
# будем работать с локальной копией истории покупок
def can_purchase (amount, history, limit, do_print = False):
    local_copy = history.copy()
    local_copy.append(amount)
    if do_print:
        print(local_copy)
    return sum(local_copy) <= limit

limit = 100
history = [50, 40]


print(can_purchase(4, history, limit, do_print = True))
print(can_purchase(7, history, limit, do_print = True))

# при работе с копией результат правильный

[50, 40, 4]
True
[50, 40, 7]
True


### Модель памяти в Python
Придется немного обратиться к модели памяти в Python. Начнем с того, что память компьютера _линейна_ - это значит, что данные в ней лежат длинным сплошным списком из нулей и единиц. Никаких двумерных матриц.
Но мы уже знаем, что переменная позволяет записать некоторый объект в определенное имя, не задумываясь об устройстве памяти.
Так, мы записали в `history` список `[50, 40]`. Мы можем в него добавлять элементы, удалять их - и не думать о линейности памяти и ее внутреннем устройстве. Как же достигается эта магия?

В Python для достижения такого удобства оператор присваивания работает в две стадии (упрощенно):
1. Где-то в памяти компьютера резервируется большое место под список и в нем создается список `[50, 40]`. 
2. Где-то еще в памяти компьютера резервируется маленькое место под имя переменной и в него кладется два значения: имя переменной и адрес в памяти, где должно лежать ее фактическое значения. В адрес памяти кладется фактический адрес созданного в п.1 списка.

В результате наша переменная `history` сама по себе не является списком - она является _ссылкой_ на список. Это похоже на ссылки в Интернете: они не содержат в себе сам сайт, но знают его адрес и по ним можно этот сайт открыть.

Теперь разберем наш пример.
Когда мы на входе функции принимали аргумент `history`, мы по факту принимали указатель на список, который уже заранее был создан.
Вызывая `.append()`, мы изменяли список "на месте" - добавляли элемент в тот же объект. После такого объект, на который ссылается `history`, изменяется навсегда - он был `[50, 40]`, а становится `[50, 40, 4]`. И следующий вызов функции уже будет получать на вход ссылку, указывающую на список `[50, 40, 4]`.

Когда же мы делали `.copy()`, то фактически создавали новый объект в другом месте памяти, куда ушли все значения из `history`, и делали все изменения в новом объекте. Когда функция `can_purchase` завершилась, копия `history` уничтожилась - **в конце выполнения функции все созданные в ней переменные уничтожаются**.

### Создать и сразу использовать
Есть третий вариант, который свободен от проблем выше. Оператор `+` не изменяет список - он создает новый, куда входят сначала элементы из списка слева, потом элементы из списка справа. Раз мы не меняем ничего, то и ошибки быть не должно.
Этот новый объект удалится, как только мы выйдем из функции.

"Создать объект и тут же применить над ним операцию, минуя присваивание" - хорошая техника в программировании. Только следите за тем, что результат расчета действительно используется только один раз.

In [13]:
def can_purchase (amount, history, limit):
    '''
    Чтобы не изменять history, создадим список с одним элементом [amount]
    и добавим его к сумме history
    После выполнения функции , все ссылки на переменные будут уничтожены, 
    а первоначальная history не будет изменена
    '''
    return sum(history + [amount]) <= limit

limit = 100
history = [50, 40]


print(can_purchase(4, history, limit))
print(can_purchase(7, history, limit))

True
True


In [14]:
b = 15
print(b, id(b)) # ячейка памяти и значение переменной
a = b
print(a, id(a))
b = 17
print(b, id(b))

15 139765750039216
15 139765750039216
17 139765750039280


## Изменяемые и неизменяемые типы данных
В примере выше мы почуяли, что `.append()` изменяет само место в памяти, где лежит список - он добавляет к нему элемент.

Такую операцию над своими данными поддерживают не все типы. Некоторые типы данных не дают себя менять - как вы их создали в памяти, такими они и останутся до конца. Такие типы данных называются **неизменяемыми** (англ. _immutable_). Их противоположность - изменяемый объект (англ. _mutable object_).

Чтобы сделать изменение в неизменяемых объектах, надо создать новый объект в памяти и положить туда все значения, которые останутся после предлагаемых изменений.

### Сложение через копирование
К примеру, строки в Python неизменяемые, но тем не менее их можно складывать:

In [15]:
sum_string = 'a' + 'b'
sum_string

'ab'

Под капотом это работает так: в памяти выделяется место под новую строку, после чего в это место сначала копируется целиком левая строка, потом правая, и ссылка на результат записывается в `sum_string`.
Поэтому складывать строки не рекомендуется - вы будете тратить лишние вычисления на копирования внутри памяти.

Обычно неизменяемые типы данных не имеют каких-либо функцию или операторов, изменяющих значения, чтобы не путать людей, но могут быть операторы `+` для их сложения (через копирование, как описано выше).

Например, кортеж (_tuple_) неизменяем - и у него нет `.append()`, `.delete()`, `.pop()`. Но складывать их можно: как и со строками, создатся копия, состоящия из объектов слева и справа.

In [20]:
(1, 3) + (2, 5, 8)

(1, 3, 2, 5, 8)

На первый вгляд такие знания кажутся тонкостью, но в реальной разработке незнание этих фактов может привести к просадке в производительности из-за необдуманных копирований.
Мы разберем задачу на это в блоке собеседований.

### Хэш
Есть еще одна причина, почему неизменяемость важна.
Для всех встроенных в Python неизменяемых объектов можно **подсчитать хэш**. Это свойство называется _hashable_, т.е. верно утверждение "tuple is hashable".

Хэш - это некая функция, которая берет на вход объект и считает одно число, причем для разных объектов это число разное.
У хэш-функции есть два главных свойства:
1. Она быстро считается.
2. При малейшем изменении объекта хэш-функция меняется лавинообразно.

Хэш-функции позволяют организовать быстрый поиск и быстрое обращение по элементу, поэтому их использует "под капотом" _словарь_ и _множество_.
Собственно, из-за этого ключом в словаре не может выступать изменяемый объект (например, `list`) - для него нельзя подсчитать хэш.
В прошлом уроке это просто проговорили, теперь же мы знаем причину.